In [26]:
from PIL import Image, ImageFont, ImageDraw

In [50]:
font_size = 37

In [51]:
font = ImageFont.truetype("arial.ttf", font_size)

In [52]:
text = "αβγδεζηθικλμνξοπρστυφχψω"

In [53]:
def toFixed(numObj, digits=3):
    return f"{numObj:.{digits}f}"

In [54]:
def get_symbol_images(text, font, font_size):
    for symbol in text:
        image = Image.new("1", (font_size, font_size), 1)
        draw = ImageDraw.Draw(image)
        symbol_width, symbol_height = draw.textsize(symbol, font=font)
        
        new_image = Image.new("1", (symbol_width, symbol_height), 1)
        new_draw = ImageDraw.Draw(new_image)
        new_draw.text((0, 0), symbol, 0, font = font)
        new_image.save("symbols/" + symbol + ".png")

In [55]:
get_symbol_images(text, font, font_size)

In [56]:
def crop(image):
    width, height = image.size
    flag = False
    left_boarder = 0
    right_boarder = width - 1
    top_boarder = 0
    bottom_boarder = height - 1
    for x in range(width):
        if (flag):
            left_boarder = x - 1
            break
        for y in range(height):
            if (image.getpixel((x,y)) == 0):
                flag = True
                break
    flag = False
    for x in range(width-1, -1, -1):
        if (flag):
            right_boarder = x + 2
            break
        for y in range(height):
            if (image.getpixel((x,y)) == 0):
                flag = True
                break
    flag = False
    for y in range(height):
        if (flag):
            top_boarder = y - 1
            break
        for x in range(width):
            if (image.getpixel((x,y)) == 0):
                flag = True
                break
    flag = False
    for y in range(height - 1, -1 , -1):
        if (flag):
            bottom_boarder = y + 2
            break
        for x in range(width):
            if (image.getpixel((x,y)) == 0):
                flag = True
                break
    area = (left_boarder, top_boarder, right_boarder, bottom_boarder)
    cropped_image = image.crop(area)
    return cropped_image

In [57]:
for symbol in text:
    image = Image.open("symbols/" + symbol + ".png")
    cp_image = crop(image)
    cp_image.save("symbols/" + symbol + ".png")

In [58]:
import pandas as pd

In [59]:
def foo(item):
    if (item == 255):
        return 1
    return 0

In [60]:
def weight_of_black(symbol):
    image = Image.open("symbols/" + symbol + ".png")
    width, height = image.size
    sum_ = 0
    for x in range(width):
        for y in range(height):
            sum_ += foo(image.getpixel((x,y)))
    return width * height - sum_, (width * height - sum_) / (width * height)

In [61]:
def coords_of_center_of_gravity(symbol):
    image = Image.open("symbols/" + symbol + ".png")
    width, height = image.size
    sum1 = 0
    sum2 = 0
    for x in range(width):
        for y in range(height):
            sum1 += (x + 1) * (1 - foo(image.getpixel((x,y))))
    for x in range(width):
        for y in range(height):
            sum2 += (y + 1) * (1 - foo(image.getpixel((x,y)))) 
    weight = weight_of_black(symbol)
    return sum1 / weight[0] , sum2 / weight[0]

In [62]:
def norm_coords_of_center_of_gravity(symbol):
    image = Image.open("symbols/" + symbol + ".png")
    width, height = image.size
    sum1 = 0
    sum2 = 0
    for x in range(width):
        for y in range(height):
            sum1 += (x + 1)* (1 - foo(image.getpixel((x,y))))
    for x in range(width):
        for y in range(height):
            sum2 += (y + 1) * (1 - foo(image.getpixel((x,y)))) 
    weight = weight_of_black(symbol)[0]
    return toFixed(((sum1 / weight) - 1) / (width - 1)),  toFixed(((sum2 / weight) - 1) / (height - 1))

In [63]:
def axial_moments_of_inertia(symbol):
    image = Image.open("symbols/" + symbol + ".png")
    width, height = image.size
    sum1 = 0
    sum2 = 0
    center = coords_of_center_of_gravity(symbol)
    for x in range(width):
        for y in range(height):
            sum1 += (1 - foo(image.getpixel((x,y)))) * ((y + 1 - center[1]) ** 2)
    for x in range(width):
        for y in range(height):
            sum2 += (1 - foo(image.getpixel((x,y)))) * ((x + 1 - center[0]) ** 2)
    return sum1, sum2

In [64]:
def norm_axial_moments_of_inertia(symbol):
    image = Image.open("symbols/" + symbol + ".png")
    width, height = image.size
    sum1 = 0
    sum2 = 0
    center = coords_of_center_of_gravity(symbol)
    for x in range(width):
        for y in range(height):
            sum1 += ((1 - foo(image.getpixel((x,y)))) * ((y + 1 - center[1]) ** 2))
    for x in range(width):
        for y in range(height):
            sum2 += ((1 - foo(image.getpixel((x,y)))) * ((x + 1 - center[0]) ** 2))
    return sum1 / ((width ** 2) + (height ** 2)), sum2 / ((width ** 2) + (height ** 2))

In [65]:
df = pd.DataFrame({'symbol' : list(text),
                   'A' : [weight_of_black(symbol)[0] for symbol in text],
                   'B' : [toFixed(weight_of_black(symbol)[1]) for symbol in text],
                   'C' : [toFixed(coords_of_center_of_gravity(symbol)[0]) for symbol in text],
                   'D' : [toFixed(coords_of_center_of_gravity(symbol)[1]) for symbol in text],
                   'E' : [norm_coords_of_center_of_gravity(symbol)[0] for symbol in text],
                   'F' : [norm_coords_of_center_of_gravity(symbol)[1] for symbol in text],
                   'G' : [toFixed(axial_moments_of_inertia(symbol)[0]) for symbol in text],
                   'H' : [toFixed(axial_moments_of_inertia(symbol)[1]) for symbol in text],
                   'I' : [toFixed(norm_axial_moments_of_inertia(symbol)[0]) for symbol in text],
                    'J' : [toFixed(norm_axial_moments_of_inertia(symbol)[1]) for symbol in text]})

In [66]:
df

,symbol,A,B,C,D,E,F,G,H,I,J
0,α,186,0.489,10.296,10.430,0.516,0.496,7081.591,6134.737,9.306,8.061
1,β,240,0.415,7.513,16.175,0.407,0.460,19832.650,6589.962,13.725,4.561
2,γ,147,0.320,8.810,11.993,0.488,0.423,7446.993,2410.667,7.315,2.368
3,δ,228,0.469,9.566,13.439,0.504,0.478,15554.140,6088.013,14.771,5.782
4,ε,129,0.496,5.845,10.729,0.404,0.512,5093.504,1460.899,8.952,2.567
5,ζ,170,0.333,7.882,16.712,0.492,0.476,19816.876,2745.647,14.350,1.988
6,η,163,0.402,8.638,11.245,0.546,0.394,9038.184,5009.644,9.474,5.251
7,θ,247,0.494,10.709,13.150,0.511,0.506,11653.457,9033.012,11.369,8.813
8,ι,80,1.000,2.500,10.500,0.500,0.500,2660.000,100.000,6.394,0.240
9,κ,159,0.497,6.277,10.497,0.352,0.500,5311.748,3007.824,8.097,4.585


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [111]:
import matplotlib.pyplot as plt
from matplotlib import transforms

In [72]:
%matplotlib inline

In [143]:
def plotX(symbol):
    image = Image.open("pictures/" + symbol + ".png")
    width, height = image.size
    plt.axis('off')
    x = range(width)
    y = [sum([1 - foo(image.getpixel((x,y))) for y in range(height)]) for x in range(width)]
    plt.plot(x, y, color = 'green')
    plt.fill_between(x, [0 for _ in range(width)], y, facecolor = 'green')
    plt.savefig("Xprofiles/" + symbol + "profile.png")
    plt.clf()

In [144]:
for symbol in text:
    
    plotX(symbol)
    

<Figure size 432x288 with 0 Axes>

In [141]:
def plotY(symbol):
    image = Image.open("pictures/" + symbol + ".png")
    width, height = image.size
    base = plt.gca().transData
    rot = transforms.Affine2D().rotate_deg(90)
    plt.axis('off')
    x = range(height)
    y = [sum([1 - foo(image.getpixel((x,y))) for x in range(width)]) for y in range(height)]
    plt.plot(x, y, color = 'red', scalex=False, scaley=False)
    plt.fill_between(x, [0 for _ in range(height)], y, facecolor = 'red')
    plt.savefig("Yprofiles/" + symbol + "profile.png")
    plt.clf()
    image = Image.open("Yprofiles/" + symbol + "profile.png")
    image.transpose(Image.ROTATE_270).save("Yprofiles/" + symbol + "profile.png")

In [142]:
for symbol in text:
    
    plotY(symbol)
    

<Figure size 432x288 with 0 Axes>

In [67]:
df.to_csv("signs.csv",sep=";")